In [153]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.fftpack import dct, idct
from heapq import heappush, heappop
from collections import defaultdict


In [155]:
# Convert image into grayscale using cv2
def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#convert image into 8x8 blocks
def convert_to_blocks(image):
    #resize image to convert it into 8x8 blocks
    image = cv2.resize(image, (np.int32(np.ceil(image.shape[1]/8)*8), np.int32(np.ceil(image.shape[0]/8)*8)))
    blocks = []
    for i in range(0, image.shape[0], 8):
        for j in range(0, image.shape[1], 8):
            blocks.append(image[i:i+8, j:j+8])
    
    # print(blocks[0])
    return blocks

In [160]:
# Apply DCT to each block
def apply_dct(blocks):
    dct_blocks = []
    for block in blocks:
        dct_blocks.append(dct(dct(block.T, norm='ortho').T, norm='ortho'))
        # print(block)
    # print(dct(dct(blocks[0].T, norm='ortho').T, norm='ortho'))
    return dct_blocks

# Quantize the DCT coefficients
def quantize(blocks, quality):
    """
    Quantize the DCT coefficients.
    """
    quantization_matrix = np.array([
        [16, 11, 10, 16, 24, 40, 51, 61],
        [12, 12, 14, 19, 26, 58, 60, 55],
        [14, 13, 16, 24, 40, 57, 69, 56],
        [14, 17, 22, 29, 51, 87, 80, 62],
        [18, 22, 37, 56, 68, 109, 103, 77],
        [24, 35, 55, 64, 81, 104, 113, 92],
        [49, 64, 78, 87, 103, 121, 120, 101],
        [72, 92, 95, 98, 112, 100, 103, 99]
    ])
 
    quantization_matrix = quantization_matrix * (100 - quality) / 50
    quantized_blocks = []
    for block in blocks:
        quantized_block = np.round(block / quantization_matrix).astype(int)
        quantized_blocks.append(quantized_block)
    return quantized_blocks

# Perform Zigzig scanning to linearize the 8x8 block
def zigzag_scanning(block):
    zigzag = []
    for i in range(8):
        if i % 2 == 0:
            for j in range(i+1):
                zigzag.append(block[j, i-j])
        else:
            for j in range(i+1):
                zigzag.append(block[i-j, j])
    for i in range(1, 8):
        if i % 2 == 0:
            for j in range(8-i):
                zigzag.append(block[7-j, i+j])
        else:
            for j in range(8-i):
                zigzag.append(block[i+j, 7-j])
    return zigzag

In [161]:
def huffman_encoding(data):
    """
    Perform Huffman encoding on the input data.
    """
    from collections import Counter, defaultdict
    from heapq import heappush, heappop, heapify

    # Count frequency of each value in data
    frequency = Counter(data)
    
    # Create a priority queue (min-heap) from the frequency dictionary
    heap = [[weight, [symbol, ""]] for symbol, weight in frequency.items()]
    heapify(heap)
    
    while len(heap) > 1:
        lo = heappop(heap)
        hi = heappop(heap)
        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]
        heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
    
    codes = sorted(heappop(heap)[1:], key=lambda p: (len(p[-1]), p))
    huffman_dict = {symbol: code for symbol, code in codes}
    
    encoded_data = "".join([huffman_dict[d] for d in data])
    return encoded_data, huffman_dict

In [162]:
def compress_image(image, quality):
    """
    Compress the image using DCT, quantization, zigzag scanning, and Huffman encoding.
    """
    image = convert_to_grayscale(image)
    blocks = convert_to_blocks(image)
    dct_blocks = apply_dct(blocks)
    quantized_blocks = quantize(dct_blocks, quality)
    zigzag_blocks = []
    for block in quantized_blocks:
        zigzag_blocks.append(zigzag_scanning(block))
    encoded_data = ""
    for block in zigzag_blocks:
        encoded, _ = huffman_encoding(block)
        encoded_data += encoded
    return encoded_data

In [163]:
encoded_data = compress_image(cv2.imread("ex.png"), 50)

In [164]:
# decompress the encoded data and reconstruct the image using inverse steps

def huffman_decoding(encoded_data, huffman_dict):
    """
    Perform Huffman decoding on the input data.
    """
    decoded_data = ""
    code = ""
    for bit in encoded_data:
        code += bit
        if code in huffman_dict.values():
            decoded_data += list(huffman_dict.keys())[list(huffman_dict.values()).index(code)]
            code = ""
    return decoded_data


def inverse_zigzag_scanning(zigzag):
    """
    Perform inverse zigzag scanning to reconstruct the 8x8 block.
    """
    block = np.zeros((8, 8))
    i = 0
    for j in range(8):
        if j % 2 == 0:
            for k in range(j+1):
                block[k, j-k] = zigzag[i]
                i += 1
        else:
            for k in range(j+1):
                block[j-k, k] = zigzag[i]
                i += 1
    for j in range(1, 8):
        if j % 2 == 0:
            for k in range(8-j):
                block[7-k, j+k] = zigzag[i]
                i += 1
        else:
            for k in range(8-j):
                block[j+k, 7-k] = zigzag[i]
                i += 1
    return block


def dequantize(blocks, quality):
    """
    Dequantize the quantized DCT coefficients.
    """
    quantization_matrix = np.array([
        [16, 11, 10, 16, 24, 40, 51, 61],
        [12, 12, 14, 19, 26, 58, 60, 55],
        [14, 13, 16, 24, 40, 57, 69, 56],
        [14, 17, 22, 29, 51, 87, 80, 62],
        [18, 22, 37, 56, 68, 109, 103, 77],
        [24, 35, 55, 64, 81, 104, 113, 92],
        [49, 64, 78, 87, 103, 121, 120, 101],
        [72, 92, 95, 98, 112, 100, 103, 99]
    ])
    
    quantization_matrix = quantization_matrix * (100 - quality) / 50
    dequantized_blocks = []
    for block in blocks:
        dequantized_block = block * quantization_matrix
        dequantized_blocks.append(dequantized_block)
    return dequantized_blocks

def apply_idct(blocks):
    """
    Apply Inverse Discrete Cosine Transform (IDCT) to each block.
    """
    idct_blocks = []
    for block in blocks:
        idct_blocks.append(idct(idct(block.T, norm='ortho').T, norm='ortho'))
    return idct_blocks

def reconstruct_image(encoded_data, image_shape, quality):
    """
    Reconstruct the image using the encoded data.
    """
    block_size = 8
    num_blocks = (image_shape[0] // block_size) * (image_shape[1] // block_size)
    huffman_dict = defaultdict(str)
    for i in range(256):
        huffman_dict[i] = str(i)
    zigzag_blocks = []
    i = 0
    while i < len(encoded_data):
        j = i + 1
        while j < len(encoded_data) and encoded_data[i:j] in huffman_dict.values():
            j += 1
        zigzag_blocks.append(huffman_decoding(encoded_data[i:j], huffman_dict))
        i = j
    blocks = []
    for zigzag_block in zigzag_blocks:
        block = inverse_zigzag_scanning(zigzag_block)
        blocks.append(block)
    dequantized_blocks = dequantize(blocks, quality)
    idct_blocks = apply_idct(dequantized_blocks)
    reconstructed_image = np.zeros(image_shape)
    k = 0
    for i in range(0, image_shape[0], block_size):
        for j in range(0, image_shape[1], block_size):
            reconstructed_image[i:i+block_size, j:j+block_size] = idct_blocks[k]
            k += 1
    reconstructed_image = np.clip(reconstructed_image, 0, 255)
    return np.uint8(reconstructed_image)



In [165]:
# Reconstruct the image using the encoded data
image_shape = cv2.imread("ex.png").shape[:2]
reconstructed_image = reconstruct_image(encoded_data, image_shape, 50)
cv2.imwrite("reconstructed_image.png", reconstructed_image)


TypeError: can only concatenate str (not "int") to str